\---

# CSCI 3202, Fall 2024
# FINAL PROJECT

<br> 

### Your name: Gavin Hanville

<br> 

# Mancala Game Implementation

In this assignment, you are tasked with implementing various functions for a Mancala game. The game is played on a board with specific rules, and you will need to implement the core game logic by completing the `play`, `valid_move`, and `winning_eval` functions. You are provided with the `init` and `display_board` functions. The assignment is divided into two parts:

## Mancala rules to be followed 
**(there are few modifications from the original game, please read this before writing the code)**

- On every turn, select a pit and distribute its stones in a counter-clockwise direction.
    - If the last stone lands in the player's mancala, in an opponent's pit, or in one of the player's non-empty pits, no further action is taken, and the current player's turn ends.
    - If the last stone lands in the current player's empty pit and the opposite pit on the opponent's side has some stones, collect all those stones, including the one that just landed, and place them into the current player's mancala.

- If either player's pits are entirely empty, the game concludes. The player with the most stones in their mancala is declared the winner. If both players have an equal number of stones in their mancala, the game results in a tie.

## Part 1: Small Board (3 Pits of 2 Stones each) (60 points)

For the first part of the assignment, students will work on a small Mancala board. The board consists of 3 pits, each initially containing 2 stones. The students need to implement the following:

1. **play**: Implement the `play` function to allow players to take turns and make moves. The function should correctly distribute stones according to the specified game rules. The game should also switch between players after each play. **(20 points)**

2. **valid_move**: Implement the `valid_move` function to ensure that a player's chosen move is valid. It should check if the selected pit is not empty and falls within the allowed pit range. **(20 points)**

3. **winning_eval**: Implement the `winning_eval` function to determine when the game is over and which player wins. The game ends when any player's pits are all empty. The winner is the player with the most stones in their mancala. If both mancalas have the same number of stones, it's a tie. **(20 points)**

Students should test their code by playing a sequence of moves starting with Player 1: 1, 2, 3, 2, 1. \
So, it would be P1 picks pit 1, P2 picks pit 2, P1 picks pit 3...and so on.
The pits are 1-indexed when displaying and picking to make a move.

Pick the pit irrespective of its validity, and print invalid move message if chosen pit is empty or invalid.

The output generated by this experiment must match the expected output given below.

## Part 2: Play Against a Random Player (6 Pits of 4 Stones each) (40 points)

In the second part of the assignment, students will extend their implementation to a larger board. The board consists of 6 pits with 4 stones in each pit. In addition to the `play`, `valid_move`, and `winning_eval` functions, students need to create a random move generator for a random player. This random player selects a random valid pit with stones to make a move. The following steps are involved in creating the random move generator:

1. **Random Move Generator**: Define the `random_move_generator` that selects a random pit from the available non-empty pits for the random player. The random player should choose a move based on these criteria. \
Set the 'seed' value to ensure that the generated values remain consistent and reproducible when grading.

You may refer to these links: [How to generate random integers in Python](https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/#:~:text=Random%20integer%20values%20can%20be,for%20the%20generated%20integer%20values.), [How to use seed in Python random](https://www.w3schools.com/python/ref_random_seed.asp)


The objective is to play up to **10** moves in total (5 moves by student, 5 moves by random player), allowing the students to verify whether their code correctly implements the Mancala game logic. **(20 points for correct implementation of Random Move Generator)**

The output submitted should reflect the state of the board and the moves played. **(10 points for playing game, 10 points for printing out results)**

**Please make sure to call the `display_board` function after each move for both the parts and run all the cells before submitting**

In [1]:
import os
os.chdir('/home/jovyan/CSCI3202/Final Project/aima-python')
print(os.getcwd())  # Verify the change
import random as random

/home/jovyan/CSCI3202/Final Project/aima-python


In [2]:
class Board:
    """
    The Board class handles the state of the pits and mancalas, 
    and provides methods for stone distribution and move validation.
    """
    def __init__(self, pits_per_player, stones_per_pit):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.total_pits = (pits_per_player + 1) * 2  # Total pits including mancalas
        self.initialize_board()

    def initialize_board(self):
        """
        Initializes the board with the specified number of stones in each pit.
        Mancalas are initialized with zero stones.
        Board indices:
        - Player 1 pits: 0 to pits_per_player - 1
        - Player 1 mancala: pits_per_player
        - Player 2 pits: pits_per_player + 1 to total_pits - 2
        - Player 2 mancala: total_pits - 1
        """
        self.board = [self.stones_per_pit] * self.total_pits
        self.p1_mancala_index = self.pits_per_player
        self.p2_mancala_index = self.total_pits - 1

        # Set mancalas to zero
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def pit_to_board_index(self, pit_number, player):
        """
        Converts a pit number (1-based) to the corresponding board index.
        """
        if player == 1:
            return pit_number - 1  # Pits are 1-indexed
        elif player == 2:
            return self.p1_mancala_index + pit_number
        else:
            raise ValueError("Invalid player number. Should be 1 or 2.")

    def valid_move(self, pit_number, player):
        """
        Checks if the selected pit is a valid move for the current player.
        """
        index = self.pit_to_board_index(pit_number, player)
        if index < 0 or index >= self.total_pits:
            # print(f"Invalid move (Pit:{pit_number})") #Disbaled for Simulation Testing
            return False
        if self.board[index] == 0:
            # print(f"Invalid move (Pit:{pit_number})") #Disbaled for Simulation Testing
            return False
        return True

    def distribute_stones(self, pit_number, player):
        """
        Distributes stones from the selected pit according to the game rules.
        Returns the index of the last pit where a stone was placed.
        """
        index = self.pit_to_board_index(pit_number, player)
        stones = self.board[index]
        self.board[index] = 0  # Empty the selected pit

        current_index = index
        while stones > 0:
            current_index = (current_index + 1) % self.total_pits

            # Skip opponent's mancala
            if (player == 1 and current_index == self.p2_mancala_index) or \
               (player == 2 and current_index == self.p1_mancala_index):
                continue

            self.board[current_index] += 1
            stones -= 1

        return current_index

    def apply_special_rules(self, last_index, player):
        """
        Applies the special capture rule after a move.
        """
        # Check if last stone landed in an empty pit on player's side
        own_pits_start = 0 if player == 1 else self.p1_mancala_index + 1
        own_pits_end = self.p1_mancala_index - 1 if player == 1 else self.p2_mancala_index - 1

        if own_pits_start <= last_index <= own_pits_end and self.board[last_index] == 1:
            # Opposite pit index
            opposite_index = self.total_pits - 2 - last_index

            if self.board[opposite_index] > 0:
                mancala_index = self.p1_mancala_index if player == 1 else self.p2_mancala_index
                captured_stones = self.board[last_index] + self.board[opposite_index]
                self.board[last_index] = 0
                self.board[opposite_index] = 0
                self.board[mancala_index] += captured_stones
                # print(f"Player {player} captures {captured_stones} stones!") #Disbaled for Simulation Testing

    def display_board(self):
        """
        Displays the board in the original user-friendly format.
        """
        p1_pits = self.board[0:self.pits_per_player]
        p1_mancala = self.board[self.p1_mancala_index]
        p2_pits = self.board[self.p1_mancala_index + 1:self.p2_mancala_index]
        p2_mancala = self.board[self.p2_mancala_index]

        # Reverse p2_pits for display purposes
        p2_pits_display = p2_pits[::-1]

        print('P1               P2')
        print('     ____{}____     '.format(p2_mancala))
        for i in range(self.pits_per_player):
            p1_pit_stones = p1_pits[i]
            p2_pit_stones = p2_pits_display[i]
            pit_num = i + 1
            reverse_pit_num = self.pits_per_player - i
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(pit_num, p1_pit_stones, p2_pit_stones, reverse_pit_num))
            else:
                print('{} -> | {} | {} | <- {}'.format(pit_num, p1_pit_stones, p2_pit_stones, reverse_pit_num))
        print('         {}         '.format(p1_mancala))
        print()  # For better readability

    def is_game_over(self):
        """
        Checks if the game is over (one player's pits are empty).
        """
        p1_side_empty = all(stone == 0 for stone in self.board[:self.pits_per_player])
        p2_side_empty = all(stone == 0 for stone in self.board[self.p1_mancala_index + 1:self.p2_mancala_index])

        return p1_side_empty or p2_side_empty

    def collect_remaining_stones(self):
        """
        At game end, collects remaining stones from the pits into respective mancalas.
        """
        # Collect stones for Player 1
        p1_remaining = sum(self.board[:self.pits_per_player])
        self.board[self.p1_mancala_index] += p1_remaining
        for i in range(self.pits_per_player):
            self.board[i] = 0

        # Collect stones for Player 2
        p2_remaining = sum(self.board[self.p1_mancala_index + 1:self.p2_mancala_index])
        self.board[self.p2_mancala_index] += p2_remaining
        for i in range(self.p1_mancala_index + 1, self.p2_mancala_index):
            self.board[i] = 0

    def get_winner(self):
        """
        Determines the winner based on the stones in the mancalas.
        Returns 1 if Player 1 wins, 2 if Player 2 wins, 0 for a tie.
        """
        p1_score = self.board[self.p1_mancala_index]
        p2_score = self.board[self.p2_mancala_index]

        if p1_score > p2_score:
            return 1
        elif p2_score > p1_score:
            return 2
        else:
            return 0  # Tie

class Game:
    """
    The Game class handles the player turns,
    move execution, and game progression.
    """
    def __init__(self, pits_per_player=6, stones_per_pit=4, mode='PvP', random_seed=109):
        self.board = Board(pits_per_player, stones_per_pit)
        self.current_player = 1
        self.mode = mode
        random.seed(random_seed)
        self.move_count = 0 # For Random V. Random

    def switch_player(self):
        self.current_player = 2 if self.current_player == 1 else 1

    def get_player_input(self):
        """
        Prompts the current player to select a pit.
        """
        while True:
            try:
                pit_number = int(input(f"Player {self.current_player}, select a pit (1-{self.board.pits_per_player}): "))
                if 1 <= pit_number <= self.board.pits_per_player:
                    if self.board.valid_move(pit_number, self.current_player):
                        return pit_number
                    else:
                        print("Invalid move. Pit is empty or invalid.") #Disbaled for Simulation Testing
                else:
                    print(f"Please enter a number between 1 and {self.board.pits_per_player}.")
            except ValueError:
                print("Invalid input. Please enter a number.")

    def get_random_move(self):
        """
        Generates a random valid move for the current player.
        """
        valid_pits = [pit for pit in range(1, self.board.pits_per_player + 1) if self.board.valid_move(pit, self.current_player)]
        if valid_pits:
            return random.choice(valid_pits)
        else:
            return None

    def play_turn(self):
        """
        Executes a turn for the current player.
        """
        
        #print(f"Player {self.current_player}'s turn.") #Disbaled for Simulation Testing
        if self.mode == 'PvP' or (self.mode == 'PvAI' and self.current_player == 1):
            pit_number = self.get_player_input()
        else:
            pit_number = self.get_random_move()
            if pit_number is None:
                # print(f"Player {self.current_player} has no valid moves.") #Disbaled for Simulation Testing
                return
            # print(f"Player {self.current_player} selects pit {pit_number}.") #Disbaled for Simulation Testing

        last_index = self.board.distribute_stones(pit_number, self.current_player)
        self.board.apply_special_rules(last_index, self.current_player)
        self.move_count += 1

    def play_game(self):
        """
        Main game loop.
        """
        
        while not self.board.is_game_over():
            #self.board.display_board() Disbaled for Simulation Testing
            self.play_turn()
            self.switch_player()

        # Game over, collect remaining stones
        self.board.collect_remaining_stones()
        #self.board.display_board() Disabled for Simulation Testing
        winner = self.board.get_winner()
        #Disabled for Simulation Testing
        # if winner == 1:
        #     print("Player 1 wins!")
        # elif winner == 2:
        #     print("Player 2 wins!")
        # else:
        #     print("It's a tie!")
            
        return winner, self.move_count

class Mancala:
    """
    The Mancala class provides an interface to start and play the game.
    """
    def __init__(self):
        self.start_game()

    def start_game(self):
        """
        Initializes the game based on user input.
        """
        print("Welcome to Mancala!")
        while True:
            print("Select game mode:")
            print("1. Player vs Player")
            print("2. Player vs AI")
            print("3. AI vs AI")
            mode_input = input("Enter your choice (1-3): ")
            if mode_input in ['1', '2', '3']:
                break
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")

        if mode_input == '1':
            mode = 'PvP'
        elif mode_input == '2':
            mode = 'PvAI'
        else:
            mode = 'AIvAI'

        pits_per_player = 6
        stones_per_pit = 4

        self.game = Game(pits_per_player, stones_per_pit, mode)
        self.game.play_game()

In [3]:
# Game Simulations methods
def simulate_games(num_games=100):
    # Trackers
    player1_wins = 0
    player2_wins = 0
    ties = 0
    total_moves = 0
    
    # Simulate Games
    for i in range(num_games):
        game = Game(pits_per_player=6, stones_per_pit=4, mode='AIvAI', random_seed=i)
        winner, move_count = game.play_game()
        total_moves += move_count
        
        if winner == 1:
            player1_wins += 1
        elif winner == 2:
            player2_wins +=1
        else:
            ties +=1
    
    # Calculate Percentages
    player1_win_percentage = (player1_wins / num_games) * 100
    player2_win_percentage = (player2_wins / num_games) * 100
    tie_percentage = (ties / num_games) * 100
    average_moves = total_moves / num_games
    
    # Display Results
    print("+=====SIMULATION RESULTS=====+")
    print(f"Total Games Played = {num_games}")
    print(f"Player 1 Wins: {player1_wins} ({player1_win_percentage})")
    print(f"Player 2 Wins: {player2_wins} ({player2_win_percentage})")
    print(f"Ties: {ties} ({tie_percentage})")
    print(f"Average Moves per Game: {average_moves:.2f}")
    print("+============================+")
    
        

In [4]:
# Game Simulations Calls
simulate_games()

+=====SIMULATION RESULTS=====+
Total Games Played = 100
Player 1 Wins: 44 (44.0)
Player 2 Wins: 49 (49.0)
Ties: 7 (7.000000000000001)
Average Moves per Game: 44.49
+============================+


In [5]:
# Manual Testing
#mancala = Mancala()

In [6]:
import unittest
import random

In [7]:
# Test suite for the Mancala game

#Tests default game initialization
class TestMancalaDefault(unittest.TestCase):
    def test_mancala_init(self):
        # Test default game setup
        test_mancala = Mancala()
        # Since the game starts immediately in the __init__, we need to adjust the code
        # For testing purposes, we'll simulate the game initialization separately
        test_mancala.game = Game(6, 4, mode='PvP')
        self.assertEqual(test_mancala.game.board.pits_per_player, 6)
        self.assertEqual(test_mancala.game.board.stones_per_pit, 4)
        self.assertEqual(test_mancala.game.mode, 'PvP')

    def test_game_init(self):
        game = Game(6, 4, mode='PvP')
        self.assertEqual(game.board.pits_per_player, 6)
        self.assertEqual(game.board.stones_per_pit, 4)
        self.assertEqual(game.mode, 'PvP')

    def test_board_init(self):
        board = Board(6, 4)
        self.assertEqual(len(board.board), 14)  # 6 pits each + 2 mancalas
        self.assertEqual(board.board[0], 4)     # Stones in first pit
        self.assertEqual(board.board[6], 0)     # Player 1's Mancala
        self.assertEqual(board.board[13], 0)    # Player 2's Mancala

class TestStoneDistribution(unittest.TestCase):
    def test_distribute_stones(self):
        board = Board(6, 4)
        # Player 1 selects pit 1 (pit_number=1)
        board.distribute_stones(1,1)
        self.assertEqual(board.board[0], 0)   # Original pit is empty
        self.assertEqual(board.board[1], 5)   # Next pit has one more stone
        
        # Player 2 selects pit 3
        last_index = board.distribute_stones(3, 2)
        self.assertEqual(board.board[13], 1)   # Player 2's mancala has one stone
        self.assertEqual(last_index, 13)       # Last stone landed in mancala

    def test_skip_opponent_mancala(self):
        board = Board(6, 4)
        # Player 1 distributes stones and should skip Player 2's Mancala
        board.board[5] = 10  # Set pit 6 to 10 stones for testing
        last_index = board.distribute_stones(6, 1)
        self.assertNotEqual(last_index, board.p2_mancala_index)  # Ensure Player 2's Mancala is skipped

class TestMoveValidation(unittest.TestCase):
    def test_valid_move(self):
        board = Board(6, 4)
        self.assertTrue(board.valid_move(1, 1))   # Pit 1 for Player 1, valid move
        board.board[0] = 0  # Empty Pit
        self.assertFalse(board.valid_move(1, 1))  # Cannot select an empty pit
        self.assertFalse(board.valid_move(7, 1))  # Cannot select an outside pit range

class TestSpecialRules(unittest.TestCase):
    def test_capture_stones(self):
        game = Game(6, 4)
        # Set up a specific board state where the special rule can be tested
        game.current_player = 1
        # Ensure the opposite pit has stones
        game.board.board = [0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0]
        # Player 1 plays pit 3
        last_index = game.board.distribute_stones(3, game.current_player)
        game.board.apply_special_rules(last_index, game.current_player)
        # Check that stones are captured correctly
        self.assertEqual(game.board.board[game.board.p1_mancala_index], 6)  # 5 (opponent's stones) + 1 (last stone)

class TestGameEnd(unittest.TestCase):
    def test_game_end_p1_wins(self):
        game = Game(6, 4)
        # Empty Player 2's pits to trigger game over
        for i in range(game.board.p1_mancala_index + 1, game.board.p2_mancala_index):
            game.board.board[i] = 0
        game.board.collect_remaining_stones()
        self.assertTrue(game.board.is_game_over())
        self.assertEqual(game.board.get_winner(), 1)
        self.assertGreater(game.board.board[game.board.p1_mancala_index], game.board.board[game.board.p2_mancala_index])

    def test_game_end_tie(self):
        game = Game(6, 4)
        # Equalize Mancala scores to simulate a tie
        game.board.board[game.board.p1_mancala_index] = 24
        game.board.board[game.board.p2_mancala_index] = 24
        # Empty all pits
        for i in range(game.board.total_pits):
            if i != game.board.p1_mancala_index and i != game.board.p2_mancala_index:
                game.board.board[i] = 0
        self.assertTrue(game.board.is_game_over())
        self.assertEqual(game.board.get_winner(), 0)  # It's a tie

#Tests turn switching and turn mechanics 
class TestGameplay(unittest.TestCase):
    def test_turn_switch(self):
        game = Game(6, 4)
        initial_player = game.current_player
        game.switch_player()
        self.assertNotEqual(game.current_player, initial_player)
        game.switch_player()
        self.assertEqual(game.current_player, initial_player)

    def test_play_turn(self):
        game = Game(6, 4)
        # Mock user input for Player 1
        game.get_player_input = lambda: 1
        game.play_turn()
        self.assertEqual(game.board.board[0], 0)  # Pit 1 should be empty
        self.assertEqual(game.board.board[1], 5)  # Stones distributed


In [8]:
# Run Individual Tests...
# print("Running TestMancalaDefault...")
# unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestMancalaDefault))

# # Run TestStoneDistribution
print("\nRunning TestStoneDistribution...")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestStoneDistribution))

# # Run TestMoveValidation
print("\nRunning TestMoveValidation...")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestMoveValidation))

# # Run TestSpecialRules
print("\nRunning TestSpecialRules...")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestSpecialRules))

# # Run TestGameEnd
print("\nRunning TestGameEnd...")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGameEnd))

# # Run TestGameplay
print("\nRunning TestGameplay...")
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGameplay))

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK



Running TestStoneDistribution...

Running TestMoveValidation...

Running TestSpecialRules...

Running TestGameEnd...

Running TestGameplay...


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [9]:
# Building AI Player
# Coding to AIMA Python Repository Abstract Classes
from probability4e import *
from games4e import *
from copy import deepcopy
import inspect

In [10]:
inspect.getsourcelines(Game)

(['class Game:\n',
  '    """A game is similar to a problem, but it has a utility for each\n',
  '    state and a terminal test instead of a path cost and a goal\n',
  '    test. To create a game, subclass this class and implement actions,\n',
  '    result, utility, and terminal_test. You may override display and\n',
  '    successors or you can inherit their default methods. You will also\n',
  '    need to set the .initial attribute to the initial state; this can\n',
  '    be done in the constructor."""\n',
  '\n',
  '    def actions(self, state):\n',
  '        """Return a list of the allowable moves at this point."""\n',
  '        raise NotImplementedError\n',
  '\n',
  '    def result(self, state, move):\n',
  '        """Return the state that results from making a move from a state."""\n',
  '        raise NotImplementedError\n',
  '\n',
  '    def utility(self, state, player):\n',
  '        """Return the value of this final state to player."""\n',
  '        raise NotImpleme

In [11]:
from games import *
from collections import namedtuple, Counter, defaultdict
import random as random
import math
import functools 
from copy import deepcopy
cache = functools.lru_cache(10**6)

In [12]:
def play_game(game, strategies: dict, verbose=False):
    """
    Play a turn-taking game.
    `strategies` is a {player: strategy_function} dict,
    where strategy_function(game, state) returns a move.
    """
    state = game.initial  # Start from the initial game state
    while not game.is_terminal(state):  # Continue until a terminal state
        player = state.to_move  # Determine which player's turn it is
        move = strategies[player](game, state)  # Get the player's move
        if verbose:
            print(f"Player {player} chooses move: {move}")
            game.display(state)
        state = game.result(state, move)  # Apply the move to get the new state
    if verbose:
        print("Final State:")
        game.display(state)
    return state

def random_player(game, state):
    """
    A random strategy that selects a move randomly.
    """
    return random.choice(game.actions(state))


def player(search_algorithm):
    """
    A player strategy that uses a search algorithm (e.g., minimax or alphabeta).
    """
    return lambda game, state: search_algorithm(game, state)[1]

In [13]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state)

infinity = math.inf

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

# Testing with a DEPTH CUTOFF and HEURISTIC #

* Having issues with recursion taking too long...

In [14]:
def heuristic(game, state, player):
    p1_score = state.board[state.p1_mancala_index]
    p2_score = state.board[state.p2_mancala_index]
    return (p1_score - p2_score) if player == 1 else (p2_score - p1_score)


In [15]:
from math import inf

def minimax_search_cutoff(game, state, depth_limit=3):
    """Minimax search with a depth cutoff. Returns (value, move)."""
    player = state.to_move

    def max_value(state, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if depth == 0:  # Cutoff reached, return heuristic
            return heuristic(game, state, player), None
        v, move = -inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), depth - 1)
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if depth == 0:  # Cutoff reached
            return heuristic(game, state, player), None
        v, move = inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), depth - 1)
            if v2 < v:
                v, move = v2, a
        return v, move

    return max_value(state, depth_limit)


In [16]:
def alphabeta_search_cutoff(game, state, depth_limit=3):
    """Alpha-beta search with a depth cutoff. Returns (value, move)."""
    player = state.to_move

    def max_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if depth == 0:  # Cutoff reached
            return heuristic(game, state, player), None
        v, move = -inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth - 1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if depth == 0:
            return heuristic(game, state, player), None
        v, move = inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth - 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -inf, inf, depth_limit)


In [17]:
#NEW VERSION OF MANCALA THAT INHERITS FROM GAMES4E.IPYNB
class MancalaState:
    """
    Represents the state of the Mancala board.
    """
    def __init__(self, pits_per_player, stones_per_pit):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.board = self.initialize_board()
        self.total_pits = (self.pits_per_player + 1) * 2  # Including mancalas
        self.to_move = 1  # Player 1 starts

    #Further abstract board into it's own class like in Games File?
    def initialize_board(self):
        """
        Initialize the Mancala board with stones in pits.
        """
        total_pits = (self.pits_per_player + 1) * 2  # Including mancalas
        board = [self.stones_per_pit] * total_pits
        self.p1_mancala_index = self.pits_per_player
        self.p2_mancala_index = total_pits - 1
        board[self.p1_mancala_index] = 0  # Player 1's mancala
        board[self.p2_mancala_index] = 0  # Player 2's mancala
        return board

#---OTHER METHODS FROM Board Class in Games files---#
#      def __hash__(self):
#             """
#             Enable hashing for caching and comparison.
#             """
#             return hash((tuple(self.board), self.to_move))

#     def __eq__(self, other):
#         """
#         Equality check for states.
#         """
#         return self.board == other.board and self.to_move == other.to_move
#     def __missing__(self, loc):
#         x, y = loc
#         if 0 <= x < self.width and 0 <= y < self.height:
#             return self.empty
#         else:
#             return self.off
            
#     def __hash__(self): 
#         return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
#     def __repr__(self):
#         def row(y): return ' '.join(self[x, y] for x in range(self.width))
#         return '\n'.join(map(row, range(self.height))) +  '\n'
#----------------------------------------------------#


class MancalaGame(Game):
    """
    Mancala implementation adhering to the Game interface.
    """
    def __init__(self, pits_per_player=6, stones_per_pit=4):
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.initial = MancalaState(pits_per_player, stones_per_pit) # "Board

    def actions(self, state):
        """
        Return a list of valid moves for the current player.
        """
        player = state.to_move
        # Adjust ranges to exclude Mancalas
        start_index = 0 if player == 1 else state.pits_per_player + 1
        end_index = state.p1_mancala_index - 1 if player == 1 else state.p2_mancala_index - 1

        # Filter for pits with stones
        moves = [
            pit_index - start_index + 1
            for pit_index in range(start_index, end_index + 1)
            if state.board[pit_index] > 0
        ]
        return moves

    def result(self, state, move):
        new_state = deepcopy(state)
        player = state.to_move
        pit_index = self.get_pit_index(player, move)
        stones = new_state.board[pit_index]
        new_state.board[pit_index] = 0

        # Distribute stones
        current_index = pit_index
        while stones > 0:
            current_index = (current_index + 1) % len(new_state.board)
            # Skip opponent's Mancala
            if (player == 1 and current_index == new_state.p2_mancala_index) or \
               (player == 2 and current_index == new_state.p1_mancala_index):
                # print(f"Skipping opponent's Mancala at index {current_index}")
                continue
            new_state.board[current_index] += 1
            # print(f"Placed stone in pit {current_index}, stones left: {stones - 1}")
            stones -= 1
        
        # Apply special rules
        self.apply_special_rules(new_state, current_index, player)
        
         # Determine next player
        if self.another_turn(new_state, player, current_index):
            new_state.to_move = player
        else:
            new_state.to_move = 3 - player
            
        return new_state

    def is_terminal(self, state):
        """
        Check if the game has ended (all pits on one side are empty).
        """
        p1_pits = state.board[:state.pits_per_player]
        p2_pits = state.board[state.pits_per_player+1: state.p2_mancala_index]

        # All pits empty for one player
        p1_empty = all(stone == 0 for stone in p1_pits)
        p2_empty = all(stone == 0 for stone in p2_pits)

        return p1_empty or p2_empty

    def utility(self, state, player):
        """
        Compute the utility of the final state for the given player.
        """
        p1_score = state.board[state.p1_mancala_index]
        p2_score = state.board[state.p2_mancala_index]

        return p1_score - p2_score if player == 1 else p2_score - p1_score

    def get_pit_index(self, player, pit_number):
        """
        Get the board index for a given player's pit number.
        """
        if player == 1:
            return pit_number - 1
        else:
            return self.pits_per_player + pit_number

#     def apply_special_rules(self, state, last_index, player):
#         """
#         Apply the capture rule for the final landing pit.
#         """
#         own_pits_start = 0 if player == 1 else state.p1_mancala_index + 1
#         own_pits_end = state.p1_mancala_index - 1 if player == 1 else state.p2_mancala_index - 1

#         # Check if last_index is on the player's side and was empty BEFORE the last stone was placed
#         if own_pits_start <= last_index <= own_pits_end and state.board[last_index] == 1:
#             # Verify the pit was empty before the last stone landed
#             if state.board[last_index] - 1 == 0:
#                 opposite_index = state.total_pits - 2 - last_index
#                 if state.board[opposite_index] > 0:  # Opponent's pit must have stones
#                     mancala_index = state.p1_mancala_index if player == 1 else state.p2_mancala_index
#                     captured_stones = state.board[last_index] + state.board[opposite_index]
#                     state.board[last_index] = 0
#                     state.board[opposite_index] = 0
#                     state.board[mancala_index] += captured_stones

    def apply_special_rules(self, state, last_index, player):
        """
        Apply the capture rule for the final landing pit.
        """
        own_pits_start = 0 if player == 1 else state.p1_mancala_index + 1
        own_pits_end = state.p1_mancala_index - 1 if player == 1 else state.p2_mancala_index - 1

        # Ensure the last_index is on the player's side
        if own_pits_start <= last_index <= own_pits_end:
            # Store the pit state BEFORE the stone was placed
            pit_was_empty = state.board[last_index] == 1
            if pit_was_empty:
                opposite_index = state.total_pits - 2 - last_index
                if state.board[opposite_index] > 0:  # Opponent's pit must have stones
                    mancala_index = state.p1_mancala_index if player == 1 else state.p2_mancala_index
                    captured_stones = state.board[last_index] + state.board[opposite_index]
                    state.board[last_index] = 0
                    state.board[opposite_index] = 0
                    state.board[mancala_index] += captured_stones



    def another_turn(self, state, player, last_index):
        """
        Check if the player gets another turn (last stone in their mancala).
        """
        mancala_index = state.p1_mancala_index if player == 1 else state.p2_mancala_index
        return last_index == mancala_index
    
    def display_board(self, state):
        """
        Displays the board in the original user-friendly format.
        """
        p1_pits = state.board[0:self.pits_per_player]
        p1_mancala = state.board[state.p1_mancala_index]
        p2_pits = state.board[state.p1_mancala_index + 1:state.p2_mancala_index]
        p2_mancala = state.board[state.p2_mancala_index]

        # Reverse p2_pits for display purposes
        p2_pits_display = p2_pits[::-1]

        print('P1               P2')
        print('     ____{}____     '.format(p2_mancala))
        for i in range(self.pits_per_player):
            p1_pit_stones = p1_pits[i]
            p2_pit_stones = p2_pits_display[i]
            pit_num = i + 1
            reverse_pit_num = self.pits_per_player - i
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(pit_num, p1_pit_stones, p2_pit_stones, reverse_pit_num))
            else:
                print('{} -> | {} | {} | <- {}'.format(pit_num, p1_pit_stones, p2_pit_stones, reverse_pit_num))
        print('         {}         '.format(p1_mancala))
        print()  # For better readability


In [18]:
import unittest
from copy import deepcopy

class TestMancalaGame(unittest.TestCase):

    def setUp(self):
        """Set up a standard Mancala game with 4 pits per player and 3 stones per pit."""
        self.game = MancalaGame(pits_per_player=4, stones_per_pit=3)
        self.initial_state = deepcopy(self.game.initial)

    def test_actions(self):
        """Test the actions method for valid moves."""
        actions = self.game.actions(self.initial_state)
        # Expect valid moves for Player 1 (1-indexed pit numbers)
        self.assertEqual(actions, [1, 2, 3, 4], "Initial actions for Player 1 are incorrect.")

        # Simulate Player 1 taking a move and validate Player 2's actions
        state_after_move = self.game.result(self.initial_state, 1)
        actions_player_2 = self.game.actions(state_after_move)
        self.assertEqual(actions_player_2, [1, 2, 3, 4], "Actions for Player 2 are incorrect.")

    def test_result(self):
        """Test the result method for correct state transitions."""
        move = 1  # Player 1 moves from pit 1
        new_state = self.game.result(self.initial_state, move)

        # Check that stones have been distributed correctly
        expected_board = [0, 4, 4, 4, 0, 3, 3, 3, 3, 0]
        self.assertEqual(new_state.board, expected_board, "Board state after move is incorrect.")

        # Check that Player 2 is now to move
        self.assertEqual(new_state.to_move, 2, "Next player to move is incorrect.")

    def test_turn_switching(self):
        """Test switching turns between players."""
        state_after_move = self.game.result(self.initial_state, 1)
        self.assertEqual(state_after_move.to_move, 2, "Turn did not switch to Player 2.")

        # Player 2 makes a move
        state_after_player_2 = self.game.result(state_after_move, 1)
        self.assertEqual(state_after_player_2.to_move, 1, "Turn did not switch back to Player 1.")

    def test_is_terminal(self):
        """Test the is_terminal method for correctly detecting end-game states."""
        # Create a terminal state (all pits on one side are empty)
        terminal_state = deepcopy(self.initial_state)
        terminal_state.board = [0, 0, 0, 0, 10, 3, 3, 3, 3, 0]
        self.assertTrue(self.game.is_terminal(terminal_state), "Terminal state not recognized.")

        # Test a non-terminal state
        non_terminal_state = deepcopy(self.initial_state)
        self.assertFalse(self.game.is_terminal(non_terminal_state), "Non-terminal state misidentified.")

    def test_utility(self):
        """Test the utility function for correct score evaluation."""
        # Create a final state with Player 1 winning
        final_state_p1_win = deepcopy(self.initial_state)
        final_state_p1_win.board = [0, 0, 0, 0, 15, 0, 0, 0, 0, 5]
        self.assertEqual(self.game.utility(final_state_p1_win, 1), 10, "Utility calculation for Player 1 is incorrect.")
        self.assertEqual(self.game.utility(final_state_p1_win, 2), -10, "Utility calculation for Player 2 is incorrect.")

        # Create a final state with a tie
        final_state_tie = deepcopy(self.initial_state)
        final_state_tie.board = [0, 0, 0, 0, 10, 0, 0, 0, 0, 10]
        self.assertEqual(self.game.utility(final_state_tie, 1), 0, "Utility calculation for a tie is incorrect.")
        self.assertEqual(self.game.utility(final_state_tie, 2), 0, "Utility calculation for a tie is incorrect.")

    # def test_apply_special_rules(self):
    #     """Test that special rules like captures are applied correctly."""
    #     state = deepcopy(self.initial_state)
    #     state.board = [0, 1, 4, 0, 5, 3, 1, 0, 4, 6]  # Simulate a mid-game state
    #     result_state = self.game.result(state, 2)  # Player 1 plays pit 2
    #     expected_board = [0, 0, 4, 0, 5, 3, 1, 0, 4, 6]
    #     expected_board[4] += 2  # Capture stones into Player 1's Mancala
    #     self.assertEqual(result_state.board, expected_board, "Capture rule not applied correctly.")

    def test_complete_game(self):
        """Simulate a complete game to verify no infinite loops or errors."""
        strategies = {
            1: random_player,
            2: random_player
        }
        final_state = play_game(self.game, strategies, verbose=False)

        # Ensure the game ends in a terminal state
        self.assertTrue(self.game.is_terminal(final_state), "Game did not end in a terminal state.")

        # Verify that scores are calculated correctly
        p1_score = final_state.board[final_state.p1_mancala_index]
        p2_score = final_state.board[final_state.p2_mancala_index]
        self.assertGreaterEqual(p1_score + p2_score, 0, "Scores are invalid at the end of the game.")



### UNIT TESTING OUR IMPLEMENTATION ###

In [19]:
unittest.TextTestRunner(verbosity=2).run(unittest.defaultTestLoader.loadTestsFromTestCase(TestMancalaGame))

test_actions (__main__.TestMancalaGame)
Test the actions method for valid moves. ... ok
test_complete_game (__main__.TestMancalaGame)
Simulate a complete game to verify no infinite loops or errors. ... ok
test_is_terminal (__main__.TestMancalaGame)
Test the is_terminal method for correctly detecting end-game states. ... ok
test_result (__main__.TestMancalaGame)
Test the result method for correct state transitions. ... ok
test_turn_switching (__main__.TestMancalaGame)
Test switching turns between players. ... ok
test_utility (__main__.TestMancalaGame)
Test the utility function for correct score evaluation. ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>

In [20]:
import random
def random_player(game, state):
    """
    A random strategy that selects a move randomly.
    """
    return random.choice(game.actions(state))


def player(search_algorithm):
    """
    A player strategy that uses a search algorithm (e.g., minimax or alphabeta).
    """
    return lambda game, state: search_algorithm(game, state)[1]


In [21]:
def simulate_games(num_games=100, pits_per_player=6, stones_per_pit=4, strategies=None, verbose = False):
    """
    Simulate multiple games between two strategies and provide a statistics summary report.
    
    Args:
    - num_games: Number of games to simulate.
    - pits_per_player: Number of pits per player in the Mancala game.
    - stones_per_pit: Initial number of stones per pit.
    - strategies: A dictionary {player_number: strategy_function}.
      If None, defaults to {1: random_player, 2: player(alphabeta_search_cutoff)}.

    Returns:
    - None (Prints the results summary).
    """
    # Default strategies if none provided
    if strategies is None:
        strategies = {
            1: random_player,  # Player 1 uses random strategy
            2: random_player  # Player 2 uses AlphaBeta
        }

    # Trackers
    player1_wins = 0
    player2_wins = 0
    ties = 0
    total_moves = 0

    # Simulate Games
    for i in range(num_games):
        # Initialize the game
        game = MancalaGame(pits_per_player=pits_per_player, stones_per_pit=stones_per_pit)
        
        # Play the game
        move_count = 0
        state = game.initial
        turns = []
        while not game.is_terminal(state):
            player_to_move = state.to_move
            
            if verbose:
                print(f"+------------------+")
                print(f"\n Turn: Player {player_to_move}")
                game.display_board(state)
            
            # Get valid moves
            valid_moves = game.actions(state)
            
            if verbose:
                print(f"Valid Moves: {valid_moves}")
            
            turns.append(player_to_move)
            move = strategies[player_to_move](game, state)  # Get move from the strategy
            
            if verbose:
                print(f"Player {player_to_move} chooses: ({move})")
            
            state = game.result(state, move)
            
            if verbose:
                print("\nUpdated Board:")
                game.display_board(state)
                print(f"+------------------+")
            
            move_count += 1
            
        

        # Update trackers
        p1_score = state.board[state.p1_mancala_index]
        p2_score = state.board[state.p2_mancala_index]
        total_moves += move_count
        p1_turns = turns.count(1)
        p2_turns = turns.count(2)
        
        if verbose:
            print(f"FINAL GAME SCORE")
            print(f"------------------")
            print(f"P1 Score = {p1_score}")
            print(f"P1 Score = {p2_score}")
            print(f"P1 Turns = ({p1_turns})")
            print(f"P2 Turns = ({p2_turns})")
            print(f"------------------")

        if p1_score > p2_score:
            player1_wins += 1
        elif p2_score > p1_score:
            player2_wins += 1
        else:
            ties += 1

    # Calculate Percentages
    player1_win_percentage = (player1_wins / num_games) * 100
    player2_win_percentage = (player2_wins / num_games) * 100
    tie_percentage = (ties / num_games) * 100
    average_moves = total_moves / num_games

    # Display Results
    print("+=====SIMULATION RESULTS=====+")
    print(f"Total Games Played = {num_games}")
    print(f"Player 1 Wins: {player1_wins} ({player1_win_percentage:.2f}%)")
    print(f"Player 2 Wins: {player2_wins} ({player2_win_percentage:.2f}%)")
    print(f"Ties: {ties} ({tie_percentage:.2f}%)")
    print(f"Average Moves per Game: {average_moves:.2f}")
    print("+============================+")


### SIMULATIONS AND SUMMARIES ###

In [22]:
strategy_tests = [
    {"1: Random vs Random": {
        1: random_player,
        2: random_player
    }},
    {"2: Random vs AlphaBeta": {
        1: random_player,
        2: player(lambda game, state: alphabeta_search_cutoff(game, state, depth_limit=5))
    }},
    {"3: Random vs Minimax": {
        1: random_player,
        2: player(lambda game, state: minimax_search_cutoff(game, state, depth_limit=5))
    }},
    {"4: AlphaBeta vs Minimax": {
        1: player(lambda game, state: alphabeta_search_cutoff(game, state, depth_limit=5)),
        2: player(lambda game, state: minimax_search_cutoff(game, state, depth_limit=5))
    }},
    {"5: Random vs AlphaBeta (10 Plies)": {
        1: random_player,
        2: player(lambda game, state: alphabeta_search_cutoff(game, state, depth_limit=10))
    }}
]

# Iterate over the dictionary items within the list
for test_case in strategy_tests:
    for test_name, strategies in test_case.items():
        print(f"Running Test: {test_name}")
        simulate_games(num_games=100, pits_per_player=6, stones_per_pit=4, strategies=strategies, verbose = False)

Running Test: 1: Random vs Random
+=====SIMULATION RESULTS=====+
Total Games Played = 100
Player 1 Wins: 53 (53.00%)
Player 2 Wins: 43 (43.00%)
Ties: 4 (4.00%)
Average Moves per Game: 45.37
+============================+
Running Test: 2: Random vs AlphaBeta
+=====SIMULATION RESULTS=====+
Total Games Played = 100
Player 1 Wins: 0 (0.00%)
Player 2 Wins: 100 (100.00%)
Ties: 0 (0.00%)
Average Moves per Game: 35.56
+============================+
Running Test: 3: Random vs Minimax
+=====SIMULATION RESULTS=====+
Total Games Played = 100
Player 1 Wins: 0 (0.00%)
Player 2 Wins: 100 (100.00%)
Ties: 0 (0.00%)
Average Moves per Game: 35.39
+============================+
Running Test: 4: AlphaBeta vs Minimax
+=====SIMULATION RESULTS=====+
Total Games Played = 100
Player 1 Wins: 100 (100.00%)
Player 2 Wins: 0 (0.00%)
Ties: 0 (0.00%)
Average Moves per Game: 37.00
+============================+
Running Test: 5: Random vs AlphaBeta (10 Plies)
+=====SIMULATION RESULTS=====+
Total Games Played = 100
Play

In [23]:
def play_against_ai(game, ai_strategy, verbose=True):
    """
    Play a Mancala game manually against an AI opponent.
    
    Args:
    - game: The Mancala game instance.
    - ai_strategy: A callable function for the AI strategy.
    - verbose: If True, displays board state after every move.
    
    Returns:
    - None
    """
    # Initialize the game state
    state = game.initial
    while not game.is_terminal(state):
        current_player = state.to_move
        
        if current_player == 1:
            # Player 1 (human) turn
            print("\nYour Turn (Player 1):")
            game.display_board(state)
            
            # Get valid moves
            valid_moves = game.actions(state)
            print(f"Valid Moves: {valid_moves}")
            
            # Prompt player for input until a valid move is entered
            move = None
            while move not in valid_moves:
                try:
                    move = int(input("Choose a pit to move from (1-indexed): "))
                    if move not in valid_moves:
                        print("Invalid move. Please try again.")
                except ValueError:
                    print("Invalid input. Please enter a number.")
            
            # Apply the move
            state = game.result(state, move)
        
        else:
            # Player 2 (AI) turn
            print("\nAI's Turn (Player 2):")
            game.display_board(state)
            
            # Get AI move using the provided strategy
            move = ai_strategy(game, state)
            print(f"AI chooses pit {move}")
            
            # Apply the move
            state = game.result(state, move)
        
        # Display updated board state if verbose
        if verbose:
            print("\nUpdated Board:")
            game.display_board(state)
    
    # Game is over; determine and display the winner
    print("\nGame Over!")
    game.display_board(state)
    p1_score = state.board[state.p1_mancala_index]
    p2_score = state.board[state.p2_mancala_index]
    
    if p1_score > p2_score:
        print(f"Player 1 Wins! (Score: {p1_score} to {p2_score})")
    elif p2_score > p1_score:
        print(f"Player 2 (AI) Wins! (Score: {p2_score} to {p1_score})")
    else:
        print(f"It's a Tie! (Score: {p1_score} to {p2_score})")


### PLAY AGAINST AI ###

In [24]:
# Create a Mancala game instance
game = MancalaGame(pits_per_player=6, stones_per_pit=4)

# Define AI strategy (e.g., AlphaBeta with depth limit 5)
ai_strategy = player(lambda g, s: alphabeta_search_cutoff(g, s, depth_limit=5))

# Uncomment line below to start the manual play session
#play_against_ai(game, ai_strategy, verbose=True)

### GAME TREE ###

In [25]:
# Refactor our HW1 Tree and Node Class to Suit the Problem

class Node:
    def __init__(self, key, parent=None, left_child=None, right_child=None):
        self.key = key  # Key could be the move or state evaluation
        self.parent = parent
        self.left_child = left_child
        self.right_child = right_child

    def getChildren(self):
        return [self.left_child, self.right_child] if self.left_child or self.right_child else []

    def __repr__(self):
        return f"Node({self.key})"


class Tree:
    def __init__(self, rootkey):
        self.root = Node(rootkey)

    def checkTree(self, parentKey, root):
        """
        Recursively find a node with the matching parentKey.
        """
        if root is None:
            return False
        if root.key == parentKey:
            return root
        for child in root.getChildren():
            found = self.checkTree(parentKey, child)
            if found:
                return found
        return False

    def add(self, key, parentKey):
        """
        Add a new node with 'key' under the node with 'parentKey'.
        """
        parentNode = self.checkTree(parentKey, self.root)
        if parentNode:
            newNode = Node(key)
            if parentNode.left_child is None:
                parentNode.left_child = newNode
                newNode.parent = parentNode
            elif parentNode.right_child is None:
                parentNode.right_child = newNode
                newNode.parent = parentNode
        else:
            # Skip adding if parent not found
            return False

    def printTree(self):
        """
        Print the tree in a structured format.
        """
        if self.root is not None:
            self.printBranch(self.root, level=0)
        else:
            print("Tree is empty.")

    def printBranch(self, root, level):
        """
        Print a single branch of the tree recursively.
        """
        if root is None:
            return
        print(" " * (4 * level) + f"-> {root.key}")
        for child in root.getChildren():
            self.printBranch(child, level + 1)


In [26]:
def alphabeta_search_cutoff_with_tree(game, state, depth_limit=3):
    """
    Alpha-beta search with a depth cutoff and decision tree construction and visualization.
    
    Args:
        game: The game object inheriting from Game
        state: The current state of the game
        depth_limit: Maximum depth allowed to evaluate to
        
    Returns:
        A tuple (value, move) that has:
            - value: The evaluation score of the best move
            - move: Actual option to select determined by the algorithm
            
    Tree key:
        - U: Utility
        - H: Heuristic
        - V: Alphabeta-Value
    
    """
    player = state.to_move
    decision_tree = Tree("Root")  # Initialize tree with root node

    def max_value(state, alpha, beta, depth, parent_key):
        if game.is_terminal(state):
            utility = game.utility(state, player)
            decision_tree.add(f"U:{utility}", parent_key)
            return utility, None
        if depth == 0:
            heuristic_value = heuristic(game, state, player)
            decision_tree.add(f"H:{heuristic_value}", parent_key)
            return heuristic_value, None

        v, move = -float('inf'), None
        for a in game.actions(state):
            child_key = f"{parent_key}->{a}"  # Unique key for each child node
            decision_tree.add(child_key, parent_key)
            v2, _ = min_value(game.result(state, a), alpha, beta, depth - 1, child_key)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                break
        decision_tree.add(f"V:{v}", parent_key)
        return v, move

    def min_value(state, alpha, beta, depth, parent_key):
        if game.is_terminal(state):
            utility = game.utility(state, player)
            decision_tree.add(f"U:{utility}", parent_key)
            return utility, None
        if depth == 0:
            heuristic_value = heuristic(game, state, player)
            decision_tree.add(f"H:{heuristic_value}", parent_key)
            return heuristic_value, None

        v, move = float('inf'), None
        for a in game.actions(state):
            child_key = f"{parent_key}->{a}"
            decision_tree.add(child_key, parent_key)
            v2, _ = max_value(game.result(state, a), alpha, beta, depth - 1, child_key)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                break
        decision_tree.add(f"V:{v}", parent_key)
        return v, move

    value, best_move = max_value(state, -float('inf'), float('inf'), depth_limit, "Root")
    decision_tree.printTree()  # Visualize the decision tree
    return value, best_move

In [27]:
game = MancalaGame(pits_per_player=6, stones_per_pit=4)
state = game.initial

print(f"+=====Alphabeta Search Tree=====+")
value, best_move = alphabeta_search_cutoff_with_tree(game, state, depth_limit=3)
print(f"Best move: {best_move}, Value: {value}")
print(f"+===============================+")

+=====Alphabeta Search Tree=====+
-> Root
    -> Root->1
        -> Root->1->1
            -> Root->1->1->2
                -> H:1
            -> Root->1->1->3
                -> H:1
        -> Root->1->2
            -> Root->1->2->2
                -> H:1
            -> V:1
    -> Root->2
        -> Root->2->1
            -> Root->2->1->1
                -> H:0
            -> Root->2->1->3
                -> H:1
        -> Root->2->2
            -> Root->2->2->1
                -> H:0
            -> Root->2->2->3
                -> H:1
Best move: 3, Value: 1
+===============================+


In [28]:
import requests
from bs4 import BeautifulSoup

def decode_url(input_url):
    # Step 1: Retrieve Data from input_url
    response = requests.get(input_url)
    if response.status_code != 200:
        print("Failed to GET document")
        return
    
    # Step 2: Parse HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    rows = soup.find_all('tr')  # Find all rows in the table
    data = []
    
    for row in rows:
        cells = row.find_all('td')  # Find all cells in the row
        if len(cells) == 3:  # Ensure there are exactly 3 columns
            try:
                # Attempt to parse the x, y coordinates and the character
                x = int(cells[0].get_text(strip=True))
                char = cells[1].get_text(strip=True)
                y = int(cells[2].get_text(strip=True))
                data.append((char, x, y))
            except ValueError:
                # Skip rows with invalid data (e.g., headers)
                continue

    # Step 3: Determine Grid Dimensions
    if not data:
        print("No valid data found in the document.")
        return

    x_size = max(item[1] for item in data)
    y_size = max(item[2] for item in data)

    # Step 4: Build Empty Grid
    grid = [[' ' for _ in range(x_size + 1)] for _ in range(y_size + 1)]

    # Step 5: Populate Grid
    for char, x, y in data:
        grid[y][x] = char

    # Step 6: Print Grid
    for row in grid:
        print("".join(row))

In [29]:
# Example URL
url = "https://docs.google.com/document/d/e/2PACX-1vQGUck9HIFCyezsrBSnmENk5ieJuYwpt7YHYEzeNJkIb9OSDdx-ov2nRNReKQyey-cwJOoEKUhLmN9z/pub"
decode_url(url)

████████░     ████████░   ██████████░    ███████░     ██░     ██░     ███░    ███░ ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░   ████░   ████░      ██░  ██░   ██░     ██░
██░     ██░ ██░       ██░ ██░         ███░           ██░██░ ██░██░       ██░██░    ██░     ██░
████████░   ██░       ██░ ████████░   ██░           ███░ ██░██░ ██░       ███░     ██████████░
██░     ██░ ██░       ██░ ██░         ███░          ██░  █████░ ███░     ██░██░    ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░ ███░   ███░   ██░    ██░  ██░   ██░     ██░
████████░     ████████░   ██████████░    ███████░  ██░           ███░ ███░    ███░ ██░     ██░
